---
project: cleaning_data_tables
title: cleaning sampletracker wines
cdt: 2024-09-17T12:55:31
status: closed
description: "clean up of the entered sampletracker wine names with those in the ct to enable joins on those columns
conclusion: have matched 175/190 wines with entered rows in ct. Those missing are either unidentifiable or not present in the ct database. Recommendation is to add them to an excluded list until such a time as it is worth manually adding their metadata."
---


unfortunately, when I was entering the samples into the tracker, I did not have a clear data structure in place. In order to acquire the metadata, I planned on joining the entered wine names with those present in the cellartracker database. Unfortunately, fuzzy joining is not a sound foundation and it was deemed necessary to replace the original names with their verified matches. This notebook produces that result. To do this, we need to get both tables, fuzzy join on the names after cleaning, inspect the results, and replace where appropriate.

# Get the Tables


In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import duckdb as db
import polars as pl
from database_etl.definitions import DATA_DIR
from database_etl.etl.sql import ct

pl.Config.set_fmt_str_lengths(999).set_tbl_rows(15)


overwrite_sample_tracker = False
overwrite_cellar_tracker = False
raw_ct_path = DATA_DIR / "dirty_cellar_tracker.csv"
dirty_st_path = str(
    DATA_DIR / "original_sample_tracker" / "original_dirty_sample_tracker.csv"
)
download_new_file = False

con = db.connect()


move the wines with missing metadata to another table


now create a sampletracker table without the missing data rows

In [ ]:
con.sql(
    """--sql
create sequence pk_st_seq start 1;
"""
)


# Create `st`

In [ ]:
con.sql(
    f"""--sql
drop table if exists excluded;
drop table if exists matches cascade;
drop table if exists st cascade;
create or replace table st (
    pk integer primary key,
    detection varchar not null,
    wine_key varchar,
    wine varchar,
    vintage integer,
    sampler varchar,
    samplecode varchar not null unique,
    open_date varchar,
    sampled_date varchar,
    added_to_cellartracker bool,
    notes varchar,
    size float,
);
insert into st
    with
        st_loading as (
            select
                nextval('pk_st_seq') as pk,
                detection,
                cast(case when vintage is null or vintage = 'null' then '9999' else vintage end as integer) as vintage,
                trim(lower(sampler)) as sampler,
                trim(lower(samplecode)) as samplecode,
                -- replace null vintages with 9999 so that string slicing operations downstream work
                replace(
                    replace(
                        strip_accents(trim(lower(name))), '"', ''
                        ), '''', ''
                    ) as wine,
                open_date,
                sampled_date,
                case when added_to_cellartracker = 'y' then true else false end as added_to_cellartracker,
                replace(
                    replace(
                        strip_accents(trim(lower(notes))), '"', ''
                        ), '''', ''
                    ) as notes,
                size,
            from
                read_csv('{dirty_st_path}')
        ),
        st_wine_key as (
        select
            pk,
            detection,
            concat(cast(vintage as integer), ' ', trim(lower(wine))) as wine_key,
            wine,
            vintage,
            sampler,
            samplecode,
            open_date,
            sampled_date,
            added_to_cellartracker,
            notes,
            size
        from st_loading
        )
select
    pk,
    detection,
    wine_key,
    wine,
    vintage,
    sampler,
    samplecode,
    open_date,
    sampled_date,
    added_to_cellartracker,
    notes,
    size
from
    st_wine_key;
"""
)

con.sql(
    """--sql
select
    *
from
    st
limit 5
"""
).pl()


## Create CT

In [ ]:
ct.load_ct(un="OctaneOolong", pw="S74rg4z3r1", con=con, output="db")

if (
    not con.sql(
        """--sql
select * from (show tables)
"""
    )
    .df()["name"]
    .eq("ct")
    .any()
):
    raise ValueError("Execute load_ct.ipynb first")
con.sql(
    """--sql
select
    *
from
    ct
limit 3
"""
).pl()


In [ ]:
from fuzzywuzzy import fuzz, process


def build_string_lists(con: db.DuckDBPyConnection) -> tuple[list[str], list[str]]:
    left_strings: list[str] = [
        x[0] for x in con.sql("select lower(wine_key) as wine from st").fetchall()
    ]
    right_strings: list[str] = [
        x[0] for x in con.sql("select lower(wine_key) as wine from ct").fetchall()
    ]

    if not all(isinstance(x, str) for x in left_strings):
        raise TypeError("expected str")
    if not all(isinstance(x, str) for x in right_strings):
        raise TypeError("expected str")

    return left_strings, right_strings


left_strings, right_strings = build_string_lists(con=con)


def match_strings(
    left_strings: list[str], right_strings: list[str]
) -> tuple[list[str], list[int]]:
    matches = []
    scores = []
    for ls in left_strings:
        result = process.extractOne(
            query=ls, choices=right_strings, scorer=fuzz.token_set_ratio
        )
        if result:
            if len(result) == 2:
                match, score = result
                matches.append(match)
                scores.append(score)
    return matches, scores


def construct_match_df(matches: list[str], scores: list[int], pk) -> pd.DataFrame:
    match_df = pd.DataFrame(
        {
            "pk": pk,
            "left_string": left_strings,
            "match": matches,
            "score": scores,
        }
    )
    return match_df


def get_st_pk(con=con):
    return con.sql("select pk from st").df()["pk"]


def match_st_ct_wine_keys(con=con) -> pd.DataFrame:
    pk = get_st_pk(con=con)
    left_strings, right_strings = build_string_lists(con=con)
    matches, scores = match_strings(
        left_strings=left_strings, right_strings=right_strings
    )

    return construct_match_df(matches=matches, scores=scores, pk=pk)


match_df = match_st_ct_wine_keys(con=con)
match_df


In [ ]:
con.sql(
    """--sql
create or replace table matches (
    pk integer primary key references st(pk),
    left_string varchar not null,
    match varchar not null,
    score integer not null,
    verified bool default false,
    );
"""
)

con.sql(
    """--sql
insert into matches
    select
        pk,
        left_string,
        match,
        score,
        false as verified
    from
        match_df
    order by
        score desc
    on conflict do nothing
        ;
"""
)

con.sql(
    """--sql
select
    *
from
    matches
order by
    pk
limit 5
"""
).pl()


In [ ]:
con.sql(
    """--sql
select
    (select count(*) from matches) as total_count,
"""
).pl()


In [ ]:
con.sql(
    """--sql
from histogram(matches, score)
"""
).df().style.set_properties(text_align="right")


As we can see the majority are above 90.

In [ ]:
con.execute(
    """--sql
select
    score,
    count(score)*100/(select count(*) from matches) as count_perc
from
    matches
where
    score > 90
group by
    score
"""
).pl()


Now lets get rid of the 100 scores.


Now there's nothing for it but to go through each match, bracket by bracket.

the 90's:

In [ ]:
con.sql(
    """--sql
select
    *
from
    matches
where
    score > 90
"""
).pl()


90's look good.

In [ ]:
con.sql(
    """--sql
update matches
    set
        verified = true
    where
        score > 75;
select * from matches where verified = false;
"""
).pl()


It appears that anything below a score of 75 is an incorrect match. These will be added to the 'incorrected_matches' table, and as they are low interest samples, will be exluded from downtrack analyses.

Alright, so in the end we have:

In [ ]:
con.sql(
    """--sql
select
    *
from
    matches
where
    verified = true
"""
).pl()


In [ ]:
con.sql(
    """--sql
select
    *
from
    matches
where
    verified = false
"""
).pl()


as we can see, out of 146 samples, 139 have verified wine name matches, and 7 have not, and have been excluded.

In [ ]:
con.sql(
    """--sql
create or replace table excluded (
    pk integer primary key references st(pk),
    left_string varchar not null,
    match varchar not null,
    score varchar not null,
    reason varchar not null,
    );
insert into excluded
    select
        pk,
        left_string,
        match,
        score,
        'missing cellatracker entry' as reason,
    from
        matches
    where
        verified = false
        ;
select
    *
from
    excluded
"""
).pl()


Looks good. Now to replace the sample tracker wine key with the cellar tracker wine key for the verified samples.

In [ ]:
con.sql(
    """--sql
alter table st add column if not exists new_wine_key varchar;
update st
    set new_wine_key = match
    from
        matches
    where
        wine_key = left_string
    and
        verified = true
    and
        matches.pk = st.pk;
alter table st add column if not exists new_wine varchar;
update st orig
    set new_wine = new.new_wine_key[6:]
    from
        st new
    where
        new.pk = orig.pk
    ;
select
    wine_key,
    new_wine_key,
    wine,
    new_wine,
from
    st
limit 5
"""
).pl()


Finally, a we've matched on `vintage` + `wine`, we should verify if the matched vintage strings equal the `st.vintage` field:

In [ ]:
con.sql(
    """--sql
select
    bool_and(cast(new_wine_key[0:4] as integer) = vintage) all_vintages_equal,
from
    st
"""
).pl()


Ok, looks good to me. Can fully replace the fields now.

In [ ]:
con.sql(
    """--sql
select
    wine_key,
    new_wine_key
from
    st
"""
).pl()


In [ ]:
con.sql(
    """--sql
select
    *
from
    st
where
    new_wine_key is null
"""
).pl()


In [ ]:
con.sql(
    """--sql
select
    (select count(*) from st) as total_count,
    (select count(*) from st where new_wine_key is null) as null_count,
    (select count(*) from st where new_wine_key is not null) as not_null_count;
"""
).pl()


In [ ]:
con.sql(
    """--sql
update st as orig
    set wine_key = (
    select
        coalesce(new_wine_key, wine_key)
    from
        st as new
    where
        orig.pk = new.pk
        );
update st as orig
    set wine = (
    select
        wine_key[6:]
    from
        st as new
    where
        orig.pk = new.pk
    );
select
    wine_key,
    wine,
    vintage,
from
    st
"""
).pl()


In [ ]:
con.sql(
    """--sql
select * from st limit 3
"""
).pl()


In [ ]:
con.sql(
    """--sql
select
    pk, wine_key, added_to_cellartracker
from
    st
where
    added_to_cellartracker != 'y'
"""
).pl()


In [ ]:
con.sql(
    """--sql
select
    *
from
    excluded
"""
).pl()


only 6 samples are not "added to cellartracker", due to them not being present there, however 7 samples were excluded. Indicates that 1 sample was added to cellartracker, but the match wasnt able to be made. So which sample is present in the subset added to cellartracker, but also excluded?

## Correct 158 'added_to_cellartracker'


In [ ]:
con.sql(
    """--sql
select
    pk, wine_key, match, score
from
    excluded
join
    st
using
    (pk)
where
    st.added_to_cellartracker = 'y'
"""
).pl()


1001 totti's vino bianco..

In [ ]:
con.sql(
    """--sql
select
    *
from
    ct
where
    wine like '%tott%'
"""
).pl()


so that's incorrect. Time to correct it.

In [ ]:
con.sql(
    """--sql
select * from excluded
"""
).pl()


In [ ]:
con.sql(
    """--sql
update st
    set added_to_cellartracker = 'n'
    where
        pk = 158;
select
    pk, wine_key, added_to_cellartracker
from
    st
where
    added_to_cellartracker = 'n'
"""
).pl()


## Correct 14, 18, 27, 155 'added_to_cellartracker'


The named samples are incorrectly stated to be added to the cellartracker, but they arnt. Correct that field.


In [ ]:
def diff_st_ct_but_added_to_cellartracker_true(
    con: db.DuckDBPyConnection,
) -> pl.DataFrame:
    """
    anti join ct and st on wine, vintage where 'added_to_cellartracker' = true
    """
    return con.sql(
        """--sql
        select
            *
        from
            st
        anti join
            ct
        on
            st.vintage = ct.vintage
        and
            st.wine = ct.wine
        where
            st.added_to_cellartracker = true
        """
    ).pl()


diff_st_ct_but_added_to_cellartracker_true(con=con)


In [ ]:
con.sql(
    """--sql
update st
    set added_to_cellartracker = false
    where
        samplecode in ['14','18','27','155']
"""
)

assert diff_st_ct_but_added_to_cellartracker_true(con=con).is_empty()


## Output ST


In [ ]:
from database_etl.definitions import DATA_DIR

sampletracker_out_path = DATA_DIR / "dirty_sample_tracker_names_corrected.parquet"


def output_name_corrected_st(con: db.DuckDBPyConnection, outpath: str) -> None:
    con.sql(
        """--sql
    select
        *
    from
        st
    limit 3
    """
    ).pl().pipe(display)
    
    overwrite_sample_tracker = True
    if overwrite_sample_tracker:
        con.sql(
            f"""--sql
        copy (
            select
                detection,
                sampler,
                samplecode,
                vintage,
                wine,
                open_date,
                sampled_date,
                added_to_cellartracker,
                notes,
                size,
            from
            st
        ) to '{outpath}' (FORMAT PARQUET)
        """
        )
    print(f"st written to {outpath}")


output_name_corrected_st(con=con, outpath=str(sampletracker_out_path))


and the cleaned cellartracker..

## Output CT


In [ ]:
con.sql(
    """--sql
select
    *
from
    ct
limit 3
"""
).pl().pipe(display)

if overwrite_cellar_tracker:
    con.sql(
        """--sql
        copy (
            select
                pk,
                size,
                vintage,
                wine,
                locale,
                country,
                region,
                subregion,
                appellation,
                producer,
                type,
                color,
                category,
                varietal
            from
                ct
        ) to '/Users/jonathan/mres_thesis/database_etl/data/clean_cellar_tracker.parquet' (format parquet)
        """
    )


# Results

In the end we have ended up with a cleaned sample tracker table that is able to join to the metadata in cellar tracker:

In [ ]:
con.sql(
    """--sql
select
    'total st count' as table,
    (select count(*) from st) as count
union
select
    'inner join st to ct' as table,
    count(*) as count
from
    st
inner join
    ct
on
    st.wine = ct.wine
and
    st.vintage = ct.vintage
union
select
    'entries in excluded table' as table,
    (select count(*) from excluded) as count
"""
).pl().pipe(display)
con.close()
del con


out of 190 entries, 175 have corresponding cellartracker metadata, and 15 are missing entries.

The sample tracker names have been cleaned up, and the original has been updated, enabling joins between ct and st. I Have elected to keep the original locally in this dir. Rather than using the code above as a basis, I will keep it isolated and recreate the tables in a core database, including the `excluded` table, which will then be based on an anti join between st and ct on the vintage + wine.